In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "NF-UNSW-NB15-v3.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
0    2151027
1      91904
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
Backdoor,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
Benign,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522,...,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522
DoS,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
Exploits,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155,...,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155
Fuzzers,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231,...,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231
Generic,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656,...,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656
Reconnaissance,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,...,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Shellcode,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143


In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,123,123,123,123,123,123,123,123,123,123,...,123,123,123,123,123,123,123,123,123,123
Backdoor,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
Benign,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103,...,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103
DoS,505,505,505,505,505,505,505,505,505,505,...,505,505,505,505,505,505,505,505,505,505
Exploits,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882,...,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882
Fuzzers,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559,...,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559
Generic,476,476,476,476,476,476,476,476,476,476,...,476,476,476,476,476,476,476,476,476,476
Reconnaissance,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
Shellcode,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159


In [16]:
# X = data.drop(columns=["Attack", "Label"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X = test_data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = test_data[["Attack", "Label"]]
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
287406,59.166.0.8,149.171.126.4,4.732152e-09,0.000000e+00,0.003486,0.000005,0.000317,0.000004,1.505900e-09,1.611550e-09,...,2.249372e-06,0.001493,0.002850,2.127056e-08,2.097309e-08,4.396601e-09,4.397286e-09,0.0,4.240589e-09,"[4.732152333835413e-09, 0.0, 0.003485589444774..."
610356,59.166.0.9,149.171.126.6,6.932495e-10,7.976222e-09,0.000215,0.000002,0.000128,0.000002,9.621527e-10,9.621527e-10,...,0.000000e+00,0.000000,0.000000,4.010061e-09,4.009943e-09,1.969528e-08,1.969835e-08,0.0,1.899640e-08,"[6.932494894241552e-10, 7.97622160421992e-09, ..."
1456518,59.166.0.2,149.171.126.2,6.381741e-07,2.401223e-07,0.060366,0.000354,0.038930,0.000379,2.030844e-07,2.173323e-07,...,0.000000e+00,0.183020,0.183020,4.321202e-07,4.840666e-07,5.929219e-07,5.930143e-07,0.0,5.718822e-07,"[6.381740567526723e-07, 2.4012234891461607e-07..."
2204716,59.166.0.0,149.171.126.0,1.390953e-08,2.334703e-08,0.000405,0.000006,0.000476,0.000004,4.426392e-09,4.736937e-09,...,0.000000e+00,0.002792,0.003191,4.531849e-09,4.766241e-09,1.292322e-08,1.292524e-08,0.0,1.246465e-08,"[1.3909530687793818e-08, 2.3347026113255993e-0..."
1090799,59.166.0.1,149.171.126.3,4.602189e-09,0.000000e+00,0.000213,0.000003,0.001485,0.000003,1.464542e-09,1.567291e-09,...,9.114981e-07,0.001980,0.001320,8.530939e-11,8.530939e-11,4.275853e-09,4.276519e-09,0.0,4.124125e-09,"[4.602188564242696e-09, 0.0, 0.000212743645752..."


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
print(train_g) 
train_g = train_g.to_directed()
print(train_g) 
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

MultiGraph with 40 nodes and 157006 edges
MultiDiGraph with 40 nodes and 314010 edges


In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v2.pkl'))

/tmp/ipykernel_70254/3368500081.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_UNSW_v2.pkl'))


<All keys matched successfully>

In [32]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [33]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [34]:
# Convert to GPU
test_g = test_g.to('cuda')

In [35]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [36]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [37]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,-0.118808,-0.048968,-0.004051,0.272247,0.072699,0.032747,-0.009545,-0.019799,-0.148869,0.021905,...,-0.000161,-0.035921,-0.071515,0.006090,-0.016855,0.039446,-0.214819,0.036549,Benign,0
1,-0.118808,-0.048968,-0.004051,0.272247,0.072699,0.032747,-0.009545,-0.019799,-0.148869,0.021905,...,-0.000161,-0.035921,-0.071515,0.006090,-0.016855,0.039446,-0.214819,0.036549,Benign,0
2,-0.118808,-0.048968,-0.004051,0.272247,0.072699,0.032747,-0.009545,-0.019799,-0.148869,0.021905,...,-0.000161,-0.035921,-0.071515,0.006090,-0.016855,0.039446,-0.214819,0.036549,Benign,0
3,-0.118808,-0.048968,-0.004051,0.272247,0.072699,0.032747,-0.009545,-0.019799,-0.148869,0.021905,...,-0.000161,-0.035921,-0.071515,0.006090,-0.016855,0.039446,-0.214819,0.036549,Benign,0
4,-0.118808,-0.048968,-0.004051,0.272247,0.072699,0.032747,-0.009545,-0.019799,-0.148869,0.021905,...,-0.000161,-0.035921,-0.071515,0.006090,-0.016855,0.039446,-0.214819,0.036549,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314005,-0.158872,-0.016249,0.023708,0.271702,0.052194,0.014024,-0.005204,0.053425,-0.164214,0.017351,...,0.013105,-0.056840,-0.051101,0.040752,-0.047410,0.056726,-0.163834,0.041188,Benign,0
314006,-0.158872,-0.016249,0.023708,0.271702,0.052194,0.014024,-0.005204,0.053425,-0.164214,0.017351,...,0.013105,-0.056840,-0.051101,0.040752,-0.047410,0.056726,-0.163834,0.041188,Benign,0
314007,-0.159557,-0.015161,0.026587,0.274558,0.053395,0.013224,-0.006234,0.052522,-0.165495,0.017346,...,0.011959,-0.057943,-0.054089,0.042515,-0.046254,0.058957,-0.169275,0.042189,Benign,0
314008,-0.159441,-0.016474,0.027997,0.276539,0.051932,0.013715,-0.008325,0.052812,-0.168176,0.019330,...,0.014342,-0.056467,-0.054632,0.044285,-0.045261,0.060711,-0.170982,0.040252,Benign,0


# Embeddings CBLOF  Embeddings

In [38]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [39]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [40]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-0.118762,-0.049244,-0.003754,0.272390,0.072759,0.032789,-0.009600,-0.019717,-0.149062,0.021961,...,-0.003883,-0.022010,0.000083,-0.035993,-0.071535,0.006265,-0.016760,0.040028,-0.214892,0.036440
1,-0.118762,-0.049244,-0.003754,0.272390,0.072759,0.032789,-0.009600,-0.019717,-0.149062,0.021961,...,-0.003883,-0.022010,0.000083,-0.035993,-0.071535,0.006265,-0.016760,0.040028,-0.214892,0.036440
2,-0.118762,-0.049244,-0.003754,0.272390,0.072759,0.032789,-0.009600,-0.019717,-0.149062,0.021961,...,-0.003883,-0.022010,0.000083,-0.035993,-0.071535,0.006265,-0.016760,0.040028,-0.214892,0.036440
3,-0.118762,-0.049244,-0.003754,0.272390,0.072759,0.032789,-0.009600,-0.019717,-0.149062,0.021961,...,-0.003883,-0.022010,0.000083,-0.035993,-0.071535,0.006265,-0.016760,0.040028,-0.214892,0.036440
4,-0.118762,-0.049244,-0.003754,0.272390,0.072759,0.032789,-0.009600,-0.019717,-0.149062,0.021961,...,-0.003883,-0.022010,0.000083,-0.035993,-0.071535,0.006265,-0.016760,0.040028,-0.214892,0.036440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134570,-0.159378,-0.016303,0.027551,0.276799,0.051552,0.013121,-0.008205,0.053251,-0.168980,0.019680,...,-0.028488,0.007043,0.013775,-0.056935,-0.055735,0.044370,-0.045558,0.060698,-0.171577,0.040351
134571,-0.159466,-0.016515,0.028057,0.276516,0.051991,0.013797,-0.008344,0.052740,-0.168077,0.019285,...,-0.028368,0.006695,0.014420,-0.056388,-0.054507,0.044275,-0.045218,0.060710,-0.170910,0.040243
134572,-0.159466,-0.016515,0.028057,0.276516,0.051991,0.013797,-0.008344,0.052740,-0.168077,0.019285,...,-0.028368,0.006695,0.014420,-0.056388,-0.054507,0.044275,-0.045218,0.060710,-0.170910,0.040243
134573,-0.159466,-0.016515,0.028057,0.276516,0.051991,0.013797,-0.008344,0.052740,-0.168077,0.019285,...,-0.028368,0.006695,0.014420,-0.056388,-0.054507,0.044275,-0.045218,0.060710,-0.170910,0.040243


In [41]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [42]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
287406,4.732152e-09,0.000000e+00,0.003486,0.000005,0.000317,0.000004,1.505900e-09,1.611550e-09,0.000000e+00,2.530544e-06,...,0.001493,0.002850,2.127056e-08,2.097309e-08,4.396601e-09,4.397286e-09,0.000000,4.240589e-09,Benign,0
610356,6.932495e-10,7.976222e-09,0.000215,0.000002,0.000128,0.000002,9.621527e-10,9.621527e-10,4.168656e-09,4.198509e-07,...,0.000000,0.000000,4.010061e-09,4.009943e-09,1.969528e-08,1.969835e-08,0.000000,1.899640e-08,Benign,0
1456518,6.381741e-07,2.401223e-07,0.060366,0.000354,0.038930,0.000379,2.030844e-07,2.173323e-07,0.000000e+00,7.533152e-03,...,0.183020,0.183020,4.321202e-07,4.840666e-07,5.929219e-07,5.930143e-07,0.000000,5.718822e-07,Benign,0
2204716,1.390953e-08,2.334703e-08,0.000405,0.000006,0.000476,0.000004,4.426392e-09,4.736937e-09,0.000000e+00,1.101955e-06,...,0.002792,0.003191,4.531849e-09,4.766241e-09,1.292322e-08,1.292524e-08,0.000000,1.246465e-08,Benign,0
1090799,4.602189e-09,0.000000e+00,0.000213,0.000003,0.001485,0.000003,1.464542e-09,1.567291e-09,0.000000e+00,1.002648e-06,...,0.001980,0.001320,8.530939e-11,8.530939e-11,4.275853e-09,4.276519e-09,0.000000,4.124125e-09,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225204,1.960806e-09,1.118412e-09,0.000157,0.000002,0.000195,0.000002,2.721379e-09,2.721379e-09,1.179074e-08,1.187518e-06,...,0.000000,0.000000,1.134216e-08,1.134183e-08,4.233212e-08,4.713307e-10,0.000071,5.372992e-08,Benign,0
1161575,3.553645e-09,0.000000e+00,0.000195,0.000003,0.001865,0.000003,1.130867e-09,1.210206e-09,0.000000e+00,9.853564e-07,...,0.001936,0.001019,1.647801e-10,1.647801e-10,3.301660e-09,3.302174e-09,0.000000,3.184501e-09,Benign,0
1927342,7.894386e-07,0.000000e+00,0.030020,0.000344,0.067420,0.000375,2.512209e-07,2.688460e-07,0.000000e+00,9.662692e-03,...,0.203761,0.203761,6.386150e-08,6.386150e-08,7.334604e-07,7.335747e-07,0.000000,7.074338e-07,Benign,0
844639,2.088732e-06,0.000000e+00,0.042651,0.000786,0.060978,0.000869,5.627404e-07,5.628146e-07,0.000000e+00,4.223766e-02,...,0.599022,0.539120,1.482757e-07,1.751208e-07,1.940623e-06,1.940925e-06,0.000000,1.339281e-06,Benign,0


In [43]:
df_raw_test

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
2178991,1.050018e-06,0.000000e+00,0.051721,0.000936,0.072787,0.001040,4.458244e-07,4.470929e-07,0.000000e+00,0.020609,...,0.542037,0.421585,8.162408e-08,8.515842e-08,9.755624e-07,9.757144e-07,0.000000,9.409448e-07,Benign,0
2117342,7.500045e-09,0.000000e+00,0.000455,0.000007,0.009613,0.000010,2.386719e-09,2.554165e-09,0.000000e+00,0.000011,...,0.006238,0.001936,6.067149e-10,6.067149e-10,6.968225e-09,6.969311e-09,0.000000,6.720960e-09,Benign,0
190701,6.821420e-07,2.566659e-07,0.056635,0.000378,0.041612,0.000405,2.170762e-07,2.323058e-07,0.000000e+00,0.007917,...,0.195630,0.195630,9.074067e-07,9.074067e-07,6.337722e-07,6.338709e-07,0.000000,6.112829e-07,Benign,0
522675,6.029956e-07,1.539932e-06,0.018870,0.000143,0.121434,0.000215,1.918896e-07,2.053522e-07,0.000000e+00,0.012002,...,0.121052,0.103759,9.377919e-09,1.125174e-08,5.602379e-07,5.603251e-07,0.000000,5.403580e-07,Benign,0
976217,3.997212e-07,1.504010e-07,0.034517,0.000222,0.024384,0.000238,1.272022e-07,1.361264e-07,0.000000e+00,0.002708,...,0.114635,0.114635,8.702843e-07,8.702843e-07,3.713775e-07,3.714353e-07,0.000000,3.581992e-07,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739489,3.668466e-09,0.000000e+00,0.000224,0.000004,0.002239,0.000004,1.167406e-09,1.249308e-09,0.000000e+00,0.000001,...,0.002209,0.001052,1.701043e-10,1.701043e-10,3.408339e-09,3.408870e-09,0.000000,3.287395e-09,Benign,0
1501636,1.793067e-09,1.022736e-09,0.000159,0.000002,0.000193,0.000002,2.488576e-09,2.488576e-09,1.078209e-08,0.000001,...,0.000000,0.000000,1.037189e-08,1.037159e-08,3.818823e-08,4.310103e-10,0.000065,4.913355e-08,Benign,0
346346,2.400977e-07,7.342982e-09,0.069504,0.000485,0.081972,0.000485,7.640563e-08,8.176608e-08,0.000000e+00,0.004308,...,0.240999,0.172142,2.448536e-07,2.448536e-07,2.230727e-07,2.231074e-07,0.000000,2.151570e-07,Benign,0
1722280,6.428059e-07,0.000000e+00,0.013126,0.000242,0.018740,0.000267,1.731830e-07,1.732058e-07,0.000000e+00,0.002393,...,0.184349,0.165914,4.563174e-08,5.389329e-08,5.972253e-07,5.973183e-07,0.000000,4.121629e-07,Benign,0


In [44]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [46]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:36<00:00,  2.68s/it]

{'n_estimators': 5, 'con': 0.001}
0.6503767410055241
              precision    recall  f1-score   support

           0     1.0000    0.8586    0.9239    129059
           1     0.2321    1.0000    0.3768      5516

    accuracy                         0.8644    134575
   macro avg     0.6161    0.9293    0.6504    134575
weighted avg     0.9685    0.8644    0.9015    134575



In [47]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:58<00:00,  3.30s/it]

{'n_estimators': 2, 'con': 0.04}
0.9937861202229019
              precision    recall  f1-score   support

           0     1.0000    0.9990    0.9995    129059
           1     0.9765    1.0000    0.9881      5516

    accuracy                         0.9990    134575
   macro avg     0.9882    0.9995    0.9938    134575
weighted avg     0.9990    0.9990    0.9990    134575



In [48]:
###  CBLOF RAW

In [49]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:09,  3.80it/s]

2


  6%|▌         | 2/36 [00:00<00:08,  4.20it/s]

2


  8%|▊         | 3/36 [00:00<00:07,  4.29it/s]

...

3


 33%|███▎      | 12/36 [00:04<00:10,  2.37it/s]

3


100%|██████████| 36/36 [00:18<00:00,  1.94it/s]

{'n_estimators': 10, 'con': 0.04}
0.752985354842935
              precision    recall  f1-score   support

           0     0.9874    0.9589    0.9729     64531
           1     0.4256    0.7132    0.5331      2758

    accuracy                         0.9488     67289
   macro avg     0.7065    0.8360    0.7530     67289
weighted avg     0.9644    0.9488    0.9549     67289



In [50]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:07,  4.57it/s]

2


  6%|▌         | 2/36 [00:00<00:07,  4.67it/s]

2


  8%|▊         | 3/36 [00:00<00:07,  4.45it/s]

...

3
3


 33%|███▎      | 12/36 [00:03<00:05,  4.11it/s]

3


100%|██████████| 36/36 [00:19<00:00,  1.89it/s]

benign only
{'n_estimators': 10}
0.7429418415310767
              precision    recall  f1-score   support

           0     0.9817    0.9706    0.9761     64531
           1     0.4562    0.5776    0.5098      2758

    accuracy                         0.9545     67289
   macro avg     0.7190    0.7741    0.7429     67289
weighted avg     0.9602    0.9545    0.9570     67289



In [51]:
# HBOS  Embeddings

In [52]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [53]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:56<00:00,  4.90s/it]

{'n_estimators': 10, 'con': 0.001}
0.710550501048881
              precision    recall  f1-score   support

           0     1.0000    0.9042    0.9497    129059
           1     0.3084    1.0000    0.4714      5516

    accuracy                         0.9081    134575
   macro avg     0.6542    0.9521    0.7106    134575
weighted avg     0.9717    0.9081    0.9301    134575



In [54]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:11<00:00,  3.66s/it]

{'n_estimators': 20, 'con': 0.04}
0.9937861202229019
              precision    recall  f1-score   support

           0     1.0000    0.9990    0.9995    129059
           1     0.9765    1.0000    0.9881      5516

    accuracy                         0.9990    134575
   macro avg     0.9882    0.9995    0.9938    134575
weighted avg     0.9990    0.9990    0.9990    134575



In [55]:
##  HBOS  RAw

In [56]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:12<00:00,  2.91it/s]

{'n_estimators': 30, 'con': 0.01}
0.8525989286730036
              precision    recall  f1-score   support

           0     0.9875    0.9887    0.9881     64531
           1     0.7282    0.7063    0.7171      2758

    accuracy                         0.9772     67289
   macro avg     0.8578    0.8475    0.8526     67289
weighted avg     0.9768    0.9772    0.9770     67289



In [57]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:11<00:00,  3.10it/s]

benign only
{'n_estimators': 5}
0.8078362549428174
              precision    recall  f1-score   support

           0     0.9857    0.9811    0.9834     64531
           1     0.6012    0.6668    0.6323      2758

    accuracy                         0.9682     67289
   macro avg     0.7934    0.8239    0.8078     67289
weighted avg     0.9699    0.9682    0.9690     67289



In [58]:
##  PCA  Emb

In [59]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:08<00:00,  1.90s/it]

{'n_estimators': 5, 'con': 0.001}
0.6410838066590094
              precision    recall  f1-score   support

           0     1.0000    0.8501    0.9190    129059
           1     0.2219    1.0000    0.3632      5516

    accuracy                         0.8563    134575
   macro avg     0.6109    0.9251    0.6411    134575
weighted avg     0.9681    0.8563    0.8962    134575



In [60]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:09<00:00,  1.94s/it]

{'n_estimators': 5, 'con': 0.04}
0.9907420225998946
              precision    recall  f1-score   support

           0     0.9995    0.9990    0.9992    129059
           1     0.9765    0.9880    0.9822      5516

    accuracy                         0.9985    134575
   macro avg     0.9880    0.9935    0.9907    134575
weighted avg     0.9985    0.9985    0.9985    134575



In [61]:
##  PCA  RAw

In [62]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:13<00:00,  2.71it/s]

{'n_estimators': 5, 'con': 0.001}
0.9079572980994904
              precision    recall  f1-score   support

           0     0.9881    0.9988    0.9934     64531
           1     0.9621    0.7183    0.8225      2758

    accuracy                         0.9873     67289
   macro avg     0.9751    0.8585    0.9080     67289
weighted avg     0.9870    0.9873    0.9864     67289



In [63]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:13<00:00,  2.75it/s]

benign only
{'n_estimators': 15}
0.7820758463846016
              precision    recall  f1-score   support

           0     0.9823    0.9817    0.9820     64531
           1     0.5784    0.5859    0.5821      2758

    accuracy                         0.9655     67289
   macro avg     0.7803    0.7838    0.7821     67289
weighted avg     0.9657    0.9655    0.9656     67289



In [64]:
##  IF  Emb

In [65]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:52<00:00,  2.19s/it]

{'n_estimators': 20, 'con': 0.01}
0.7533346673661796
              precision    recall  f1-score   support

           0     1.0000    0.9283    0.9628    129059
           1     0.3735    1.0000    0.5439      5516

    accuracy                         0.9312    134575
   macro avg     0.6867    0.9642    0.7533    134575
weighted avg     0.9743    0.9312    0.9456    134575



In [66]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:54<00:00,  2.26s/it]

{'n_estimators': 100, 'con': 0.04}
0.9899797354659392
              precision    recall  f1-score   support

           0     0.9994    0.9990    0.9992    129059
           1     0.9761    0.9855    0.9808      5516

    accuracy                         0.9984    134575
   macro avg     0.9877    0.9922    0.9900    134575
weighted avg     0.9984    0.9984    0.9984    134575



In [67]:
##  IF  Raw

In [68]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:17<00:00,  1.36it/s]

{'n_estimators': 100, 'con': 0.01}
0.8812798518166669
              precision    recall  f1-score   support

           0     0.9912    0.9888    0.9900     64531
           1     0.7519    0.7944    0.7726      2758

    accuracy                         0.9808     67289
   macro avg     0.8715    0.8916    0.8813     67289
weighted avg     0.9814    0.9808    0.9811     67289



In [69]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:17<00:00,  1.36it/s]

benign only
{'n_estimators': 50}
0.8258423100422502
              precision    recall  f1-score   support

           0     0.9857    0.9858    0.9857     64531
           1     0.6669    0.6650    0.6659      2758

    accuracy                         0.9727     67289
   macro avg     0.8263    0.8254    0.8258     67289
weighted avg     0.9726    0.9727    0.9726     67289

